In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from timelab import MultiColumn, from_data, from_xy_data

In [3]:
df = pd.read_pickle('tests/test_data/multi_asset.pkl')
df = df[['LNC', 'MAS', 'CSX']]

In [4]:
panel = from_data(df, 10, 5, gap=0)

/Volumes/GoogleDrive/My Drive/Logspace/Repositories/timeLab/timelab/panel.py:47: UserWarning: Frequency is either not constant or negative.
  warnings.warn("Frequency is either not constant or negative.")
100%|██████████| 86/86 [00:00<00:00, 32989.77it/s]


In [5]:
# xdata preprocessing

xdata = df.pct_change().dropna()
xdata = MultiColumn(xdata)
xdata.head()

LNC                                                   MAS  \
                Open          High       Low     Close    Volume      Open   
Date                                                                         
2005-12-22  0.027751  1.001588e-07  0.019908 -0.004281 -0.499795  0.004950   
2005-12-23 -0.003166  1.855704e-03  0.002065  0.007290 -0.462529 -0.006896   
2005-12-27  0.011583  2.963881e-03  0.000562 -0.002226  0.584069 -0.001323   
2005-12-28 -0.001478  4.616230e-03  0.006738  0.001859 -0.541975 -0.003973   
2005-12-29 -0.003884 -5.882202e-03 -0.008181 -0.009467  0.246021 -0.002659   

                                                         CSX            \
                High       Low     Close    Volume      Open      High   
Date                                                                     
2005-12-22  0.007526  0.000331 -0.008211  0.139214  0.021713  0.003367   
2005-12-23 -0.017862 -0.011932 -0.005960  0.325702 -0.013108  0.015595   
2005-12-27  0.001984  0.006373  0.001665 -0.230328  0.035419  0.000972   
2005-12-28 -0.002970 -0.003000 -0.002328  0.259354 -0.015549 -0.010291   
2005-12-29 -0.000331 -0.001337 -0.000667 -0.240734 -0.000593  0.012360   

                                          
                 Low     Close    Volume  
Date                                      
2005-12-22  0.018466  0.007570 -0.360517  
2005-12-23 -0.009962  0.009095 -0.320835  
2005-12-27  0.011471 -0.010580  0.351240  
2005-12-28 -0.000995  0.002970 -0.270459  
2005-12-29  0.007170  0.008687  0.277498

In [6]:
# ydata preprocessing

ydata = xdata.copy()

ydata = ydata.channel_apply(lambda x: x['High'] > 0, 'Raise')
ydata = ydata.loc[xdata.index]
ydata = MultiColumn(ydata)
ydata.head()

,LNC,MAS,CSX
,Raise,Raise,Raise
Date,,,
2005-12-22,True,True,True
2005-12-23,True,False,True
2005-12-27,True,True,True
2005-12-28,True,False,False
2005-12-29,False,False,True


In [44]:
lookback = 8
horizon = 1
gap = 0

In [45]:
panel = from_xy_data(xdata, ydata, lookback=lookback, horizon=horizon, gap=gap)

/Volumes/GoogleDrive/My Drive/Logspace/Repositories/timeLab/timelab/panel.py:47: UserWarning: Frequency is either not constant or negative.
  warnings.warn("Frequency is either not constant or negative.")
100%|██████████| 91/91 [00:00<00:00, 63286.63it/s]


In [46]:
panels = panel.split_units(split_yunits=True)

100%|██████████| 3/3 [00:00<00:00, 31.17it/s]


In [10]:
stock_panel = panels[0]

In [11]:
x = stock_panel.xflat()
y = stock_panel.yflat()

x_train = x.iloc[:50]
y_train = y.iloc[:50][0] # first column

x_test = x.iloc[50:]
y_test = y.iloc[50:][0] # first column

# A split by date here (e.g. y_train = y.loc[:'2006']) could cause problems, since predictions occur in the future.
# A solution would be to map y indexes with x indexes, but we lose the prediction date information

In [12]:
# Best solution is using time panel split:
train, test = stock_panel.split_train_test(split_size=0.2)

In [13]:
x_train = train.xflat()
y_train = train.yflat()

x_test = test.xflat()
y_test = test.yflat()

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

In [22]:
rf = RandomForestClassifier(class_weight={1:0.5, 0:10})
rf.fit(x_train, y_train)
rf.predict_proba(x_test)

/var/folders/2f/p21ddq0n1m1fjxhp8r7hksv80000gn/T/ipykernel_1774/211340750.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(x_train, y_train)


array([[0.11, 0.89],
       [0.27, 0.73],
       [0.39, 0.61],
       [0.17, 0.83],
       [0.48, 0.52],
       [0.49, 0.51],
       [0.76, 0.24],
       [0.68, 0.32],
       [0.87, 0.13],
       [0.51, 0.49],
       [0.55, 0.45],
       [0.67, 0.33],
       [0.7 , 0.3 ],
       [0.45, 0.55],
       [0.34, 0.66],
       [0.46, 0.54],
       [0.65, 0.35],
       [0.62, 0.38]])

In [80]:
# pred_data = pd.DataFrame(index=x_test.index,
#                          data={'ytrue': y_test,
#                                'ypred': preds}).astype(float)

# pred_data['pred'] = pred_data.ypred > 0.5
# roc_auc_score(pred_data.ytrue, pred_data.ypred)
# print(classification_report(pred_data.ytrue, pred_data.pred))

In [47]:
panel.xframe(0)

LNC                                                   MAS  \
                Open          High       Low     Close    Volume      Open   
2005-12-22  0.027751  1.001588e-07  0.019908 -0.004281 -0.499795  0.004950   
2005-12-23 -0.003166  1.855704e-03  0.002065  0.007290 -0.462529 -0.006896   
2005-12-27  0.011583  2.963881e-03  0.000562 -0.002226  0.584069 -0.001323   
2005-12-28 -0.001478  4.616230e-03  0.006738  0.001859 -0.541975 -0.003973   
2005-12-29 -0.003884 -5.882202e-03 -0.008181 -0.009467  0.246021 -0.002659   
2005-12-30 -0.012811 -1.682683e-02 -0.013123 -0.006184  0.116100 -0.000667   
2006-01-03  0.001881  7.711192e-03 -0.002849  0.009429  0.251136  0.020014   
2006-01-04  0.009011  9.331692e-03  0.023623  0.006912  0.375622  0.017640   

                                                         CSX            \
                High       Low     Close    Volume      Open      High   
2005-12-22  0.007526  0.000331 -0.008211  0.139214  0.021713  0.003367   
2005-12-23 -0.017862 -0.011932 -0.005960  0.325702 -0.013108  0.015595   
2005-12-27  0.001984  0.006373  0.001665 -0.230328  0.035419  0.000972   
2005-12-28 -0.002970 -0.003000 -0.002328  0.259354 -0.015549 -0.010291   
2005-12-29 -0.000331 -0.001337 -0.000667 -0.240734 -0.000593  0.012360   
2005-12-30  0.002649  0.003348  0.007004  0.086383  0.004544 -0.011046   
2006-01-03  0.030714  0.007675  0.033787  1.172430  0.001376 -0.000784   
2006-01-04 -0.000968  0.016781 -0.009029 -0.343349 -0.018068 -0.016081   

                                          
                 Low     Close    Volume  
2005-12-22  0.018466  0.007570 -0.360517  
2005-12-23 -0.009962  0.009095 -0.320835  
2005-12-27  0.011471 -0.010580  0.351240  
2005-12-28 -0.000995  0.002970 -0.270459  
2005-12-29  0.007170  0.008687  0.277498  
2005-12-30  0.000791 -0.006263 -0.351358  
2006-01-03 -0.022921 -0.016151  2.486240  
2006-01-04  0.005662 -0.002603 -0.041152

In [48]:
from timelab.utils import smash_array

In [49]:
# Pipeline for one unit
panel = panels[0]

# Remove unit dimension - it is already being processed separately.
X = smash_array(panel.X)

# y is 1D so flatten is enough
y = panel.y.flatten()

In [53]:
X.shape

(91, 8, 5)

In [68]:
inputs = tf.keras.Input(shape=(X.shape[1:]))

# Convoluting on the time dimension
x = tf.keras.layers.SeparableConv1D(10, 8)(inputs)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x)
x = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(x)
outputs = tf.keras.layers.Reshape(train.y.shape[1:])(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["binary_crossentropy", 'AUC'])
model.summary()

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 8, 5)]            0         
_________________________________________________________________
separable_conv1d_1 (Separabl (None, 1, 10)             100       
_________________________________________________________________
flatten_12 (Flatten)         (None, 10)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 11        
_________________________________________________________________
reshape_12 (Reshape)         (None, 1, 1, 1)           0         
Total params: 221
Trainable params: 221
Non-trainable params: 0
____________________________________________________________

In [21]:



x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x)
x = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(x)
outputs = tf.keras.layers.Reshape(train.y.shape[1:])(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["binary_crossentropy", 'AUC'])
model.summary()

TimePanel
size                              94
lookback                           5
horizon                            1
gap                                0
num_xunits                         1
num_yunits                         1
num_xchannels                      5
num_ychannels                      1
start            2005-12-22 00:00:00
end              2006-05-17 00:00:00
Name: TimePanel, dtype: object
TimePanel
size                              94
lookback                           5
horizon                            1
gap                                0
num_xunits                         1
num_yunits                         1
num_xchannels                      5
num_ychannels                      1
start            2005-12-22 00:00:00
end              2006-05-17 00:00:00
Name: TimePanel, dtype: object
TimePanel
size                              94
lookback                           5
horizon                            1
gap                                0
num_xunits          

[<TimePanel, size 94>, <TimePanel, size 94>, <TimePanel, size 94>]

In [ ]:
import plotly.graph_objects as go
import pandas as pd

fig = go.Figure(data=[go.Candlestick(x=data.index,
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])
                ])


fig.update_layout(xaxis_rangeslider_visible=False)

shapes = []
for index in trues.index:
    shapes.append(dict(type='line', yref='paper', y0=0, y1=1, xref='x',
                 x0=index, x1=index))

fig.update_layout(shapes=shapes)

fig.show()

In [ ]:
train, test = stock_panel.split_train_test(0.2)
print(train.X.shape)
print(train.y.shape)

In [ ]:
inputs = tf.keras.Input(shape=(train.X.shape[1:]))

x = tf.keras.layers.Conv1D(2, 2)(inputs)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x)
x = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(x)
outputs = tf.keras.layers.Reshape(train.y.shape[1:])(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["binary_crossentropy", 'AUC'])
model.summary()

In [ ]:
H = model.fit(train.X, train.y, epochs=120, validation_data=(test.X, test.y), verbose=0)

In [ ]:
losses = pd.DataFrame({'train_loss': H.history['loss'], 'val_loss': H.history['val_loss']})
losses.plot()

In [ ]:
aucs = pd.DataFrame({'train_auc': H.history['auc'], 'val_auc': H.history['val_auc']})
aucs.plot()

In [ ]:
preds = model.predict(test.X)

pred_data = pd.DataFrame(index=test.yindex, 
                         data={'ytrue': test.y.flatten(), 
                               'ypred':preds.flatten()}).astype(float)

pred_data['pred'] = pred_data.ypred > 0.9

In [ ]:
pred_data.ypred.plot(kind='scatter')

In [ ]:
thresh = 0.9
pred_data[pred_data.ypred > thresh]

In [ ]:
print(classification_report(pred_data.ytrue, pred_data.ypred > thresh))

In [ ]:
# Option to add multilevel - one per column instead of one above all
# Label: if (tomorrow high - today close) > 0

In [ ]:
# xdata.loc[:, (slice(None), ['Volume'])] = xdata.loc[:, (slice(None), ['Volume'])].shift()

In [ ]:
import plotly.graph_objects as go

def pair_plot(pair, xunit, yunit, xchannels=None, ychannels=None):
    
    
    x = pair.xframe
    y = pair.yframe
    
    x = select(x, units=[xunit], channels=xchannels)
    y = select(y, units=[yunit], channels=ychannels)

    fig = go.Figure()

    for _, channel in enumerate(xchannels):
        c = random.choice(cmap1)
        fig.add_trace(go.Scatter(x=x.index, y=x[channel], name="x_" + channel,
                                 line=dict(width=2, color=c)))
        
    for _, channel in enumerate(ychannels):
        c = random.choice(cmap1)
        fig.add_trace(go.Scatter(x=y.index, y=y[channel], name="y_" + channel,
                                 line=dict(width=2, dash='dot', color=c)))

    fig.update_layout(title='', xaxis_title='Timestamps', yaxis_title='Values')
    fig.show()

In [ ]:
pair_plot(panel.pairs[0], 'MSFT', 'MSFT')

In [ ]:
xdata['QCOM']['High'].plot()